# Classification of the entire test set
In this tutorial we're going to take the model we developed in the previous tutorial, run it on the entire MNIST testing set and calculate the overall classification accuracy.

## Install PyGeNN wheel from Google Drive
Download wheel file

In [1]:
if "google.colab" in str(get_ipython()):
    !gdown 1OJH0FAR4GRsvIKkHw0NBM-9aYU2VJ1bo
    !pip install pygenn-5.1.0-cp310-cp310-linux_x86_64.whl
    %env CUDA_PATH=/usr/local/cuda

Downloading...
From: https://drive.google.com/uc?id=1OJH0FAR4GRsvIKkHw0NBM-9aYU2VJ1bo
To: /content/pygenn-5.1.0-cp310-cp310-linux_x86_64.whl
100% 8.47M/8.47M [00:00<00:00, 200MB/s]
Processing ./pygenn-5.1.0-cp310-cp310-linux_x86_64.whl
pygenn is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
env: CUDA_PATH=/usr/local/cuda


## Download pre-trained weights and MNIST test data

In [2]:
!gdown 1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
!gdown 131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF

Downloading...
From: https://drive.google.com/uc?id=1cmNL8W0QZZtn3dPHiOQnVjGAYTk6Rhpc
To: /content/weights_0_1.npy
100% 402k/402k [00:00<00:00, 47.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=131lCXLEH6aTXnBZ9Nh4eJLSy5DQ6LKSF
To: /content/weights_1_2.npy
100% 5.25k/5.25k [00:00<00:00, 19.1MB/s]


## Install MNIST package

In [3]:
!pip install mnist

## Build model
As well as the standard modules and required PyGeNN functions and classes we used in the first tutorial, also import `time.perf_counter` for measuring the performance of our classifier and `tqdm.tqdm` for drawing progress bars

In [4]:
import mnist
import numpy as np
import matplotlib.pyplot as plt
from pygenn import (create_neuron_model, create_current_source_model,
                    init_postsynaptic, init_weight_update, GeNNModel)
from time import perf_counter
from tqdm.auto import tqdm

As before, define some simulation parameters

In [5]:
TIMESTEP = 1.0
PRESENT_TIMESTEPS = 100
INPUT_CURRENT_SCALE = 1.0 / 100.0

Create very similar neuron and current source models. However, to avoid having to download every spike and count them on the CPU, here, we add an additional state variable `SpikeCount` to each neuron which gets incremented in the reset code to count spikes.

In [6]:
# Very simple integrate-and-fire neuron model
if_model = create_neuron_model(
    "if_model",
    params=["Vthr"],
    vars=[("V", "scalar"), ("SpikeCount", "unsigned int")],
    sim_code="V += Isyn * dt;",
    reset_code="""
    V = 0.0;
    SpikeCount++;
    """,
    threshold_condition_code="V >= Vthr")

cs_model = create_current_source_model(
    "cs_model",
    vars=[("magnitude", "scalar")],
    injection_code="injectCurrent(magnitude);")

Build model, load weights and create neuron, synapse and current source populations as before

In [7]:
model = GeNNModel("float", "tutorial_2")
model.dt = TIMESTEP

# Load weights
weights_0_1 = np.load("weights_0_1.npy")
weights_1_2 = np.load("weights_1_2.npy")

if_params = {"Vthr": 5.0}
if_init = {"V": 0.0, "SpikeCount":0}
neurons = [model.add_neuron_population("neuron0", weights_0_1.shape[0],
                                       if_model, if_params, if_init),
           model.add_neuron_population("neuron1", weights_0_1.shape[1],
                                       if_model, if_params, if_init),
           model.add_neuron_population("neuron2", weights_1_2.shape[1],
                                       if_model, if_params, if_init)]
model.add_synapse_population(
        "synapse_0_1", "DENSE",
        neurons[0], neurons[1],
        init_weight_update("StaticPulse", {}, {"g": weights_0_1.flatten()}),
        init_postsynaptic("DeltaCurr"))
model.add_synapse_population(
        "synapse_1_2", "DENSE",
        neurons[1], neurons[2],
        init_weight_update("StaticPulse", {}, {"g": weights_1_2.flatten()}),
        init_postsynaptic("DeltaCurr"));

current_input = model.add_current_source("current_input", cs_model,
                                         neurons[0], {}, {"magnitude": 0.0})

Run code generator to generate simulation code for model and load it into PyGeNN as before but, here, we don't want to record any spikes so no need to specify a recording buffer size.

In [8]:
model.build()
model.load()

Just like in the previous tutorial, load testing images and labels and verify their dimensions

In [9]:
mnist.datasets_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"
testing_images = mnist.test_images()
testing_labels = mnist.test_labels()

testing_images = np.reshape(testing_images, (testing_images.shape[0], -1))
assert testing_images.shape[1] == weights_0_1.shape[0]
assert np.max(testing_labels) == (weights_1_2.shape[1] - 1)

## Simulate model
In this tutorial we're going to not only inject current but also access the new spike count variable in the output population and reset the voltages throughout the model. Therefore we need to create some additional memory views

In [10]:
current_input_magnitude = current_input.vars["magnitude"]
output_spike_count = neurons[-1].vars["SpikeCount"]
neuron_voltages = [n.vars["V"] for n in neurons]

Now, we define our inference loop. We loop through all of the testing images and for each one:

1.   Copy the (scaled) image data into the current input memory view and copy it to the GPU
2.   Loop through all the neuron populations, zero their membrance voltages and copy these to the GPU
3. Zero the output spike count and copy that to the GPU
4. Simulate the model for `PRESENT_TIMESTEPS`
5. Download the spike counts from the output layer
6. If highest spike count corresponds to correct label, increment `num_correct`



In [11]:
# Simulate
num_correct = 0
start_time = perf_counter()
for i in tqdm(range(testing_images.shape[0])):
    current_input_magnitude.values = testing_images[i] * INPUT_CURRENT_SCALE
    current_input_magnitude.push_to_device()

    # Loop through all voltage variables
    for v in neuron_voltages:
        # Manually 'reset' voltage
        v.view[:] = 0.0

        # Upload
        v.push_to_device()

    # Zero spike count
    output_spike_count.view[:] = 0
    output_spike_count.push_to_device()

    for t in range(PRESENT_TIMESTEPS):
        model.step_time()

    # Download spike count from last layer
    output_spike_count.pull_from_device()

    # Find which neuron spiked the most to get prediction
    predicted_label = np.argmax(output_spike_count.values)
    true_label = testing_labels[i]

    if predicted_label == true_label:
        num_correct += 1

end_time = perf_counter()
print(f"\nAccuracy {((num_correct / float(testing_images.shape[0])) * 100.0)}%%")
print(f"Time {end_time - start_time} seconds")


  0%|          | 0/10000 [00:00<?, ?it/s]


Accuracy 97.44%%
Time 11.59845029600001 seconds
